# Data validation
In this notebook we validate and provide some basic analysis on the data we work with.

There are 2-3 data stages.
- (Collections), poorly defined counts.
- Raw, poorly defined counts.
- Freeze, contains arrangements which can contain many documents.
- Elasticsearch, contains documents split up between languages.

The input to the raw stage is specific to each collection/repository and not really considered as a data stage in the pipeline.
In the raw stage all objects are mapped to a uniform format. We do not have direct access to these files currently.
In the freeze stage, the data has been enriched and filtered. 
In the elasticsearch stage, the data is split up and subsets are selected.

To see the effect of each stage we need to look at "document" counts for a few stages:
1. Collections - Raw ? ill-defined document type
1. Raw - Freeze ? ill-defined document type
1. Freeze - Elasticsearch

Secondly, we want to verify that all documents which we use for query ranking are in the ElasticSearch index.

## 1. Count the number of documents

In [118]:
import json
import pprint
import logging
import os
import pandas as pd
from collections import defaultdict, Counter
from elasticsearch import Elasticsearch

import util

# RAW_DATA_FOLDER = 'data/raw'
FREEZE_DATA_FOLDER = 'data/freeze_04/'
ES_INDEX_NAME = 'freeze4_no_filter'
# the elasticsearch data and queries are split into languages.
LANGUAGES = ['en', 'nl']

INDIECES = {lang: f'{ES_INDEX_NAME}-{lang}' for lang in LANGUAGES}
QUERY_FILES = {lang: f'queries/queries-{lang}.json' for lang in LANGUAGES}
logging.basicConfig(format='[%(asctime)-15s][%(levelname)-7s] %(message)s',
                        level=logging.INFO)

ES_CREDENTIALS = 'es-credentials.json'
es_client = util.get_es_client(ES_CREDENTIALS)
DOC_TYPE = '_doc'

[2019-03-14 14:42:20,189][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/_cat/health [status:200 request:0.048s]


In [2]:
# def read_raw_documents():
#     folder = RAW_DATA_FOLDER
#     yield from util.read_input_documents(folder)

In [4]:
# raw_docs = list(read_raw_documents())
# print(len(raw_docs))

In [5]:
def read_processed_documents():
    folder = FREEZE_DATA_FOLDER
    return util.read_raw_documents(folder)

In [16]:
processed_docs = list(read_processed_documents())
freeze_doc_count = len(processed_docs)
freeze_doc_count

5660

Now we have the number of documents in the freeze, let's verify that we have all of them in the Elasticsearch index.

In [22]:
def get_indices(es_client):
    return es_client.cat.indices(format="json", v=True)
indieces = get_indices(es_client)
es_doc_count = 0
for index in indieces:
    if index['index'] in INDIECES:
        print(index)
        es_doc_count += int(index['docs.count'])
es_doc_count

[2019-03-14 11:57:34,430][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/_cat/indices?format=json&v=true [status:200 request:0.050s]


{'health': 'green', 'status': 'open', 'index': 'freeze4_no_filter-en', 'uuid': 'qV3MEzgKT_iVakgBSlE4nA', 'pri': '1', 'rep': '0', 'docs.count': '1532', 'docs.deleted': '35', 'store.size': '9.9mb', 'pri.store.size': '9.9mb'}
{'health': 'green', 'status': 'open', 'index': 'freeze4_no_filter-nl', 'uuid': 'UeHlzvi1TqKFQW30tgpWMw', 'pri': '1', 'rep': '0', 'docs.count': '3971', 'docs.deleted': '122', 'store.size': '28.1mb', 'pri.store.size': '28.1mb'}


5503

There is a mismatch, let's check if it is because of duplicate ids.

In [32]:
non_duplicates = Counter(doc['id'] for doc in processed_docs)
len(non_duplicates)

5503

Number of duplicate ids

In [30]:
freeze_doc_count -  len(non_duplicates)

157

Let's take a look at these duplicates.

In [89]:
duplicates = list(filter(lambda x: x[1] > 1, non_duplicates.most_common()))
duplicate_ids = [ dup[0] for dup in duplicates]

# lets first fetch all the duplicates and store them
duplicate_docs = list()
for doc in processed_docs:
    if doc['id'] in duplicate_ids:
        duplicate_docs.append(doc)
print(len(duplicate_docs))
# now we want to do a basic equality check and ignore them if they are exactly equal
duplicate_docs_df = pd.DataFrame(duplicate_docs)
duplicate_docs_df.sort_values(by=['id'])
pd.set_option('display.max_colwidth', -1)  # or 199
duplicate_docs_df


258


arrangement_collection_name  \
0    stimuleringsregeling         
1    stimuleringsregeling         
2    stimuleringsregeling         
3    stimuleringsregeling         
4    hbovpk                       
5    hbovpk                       
6    wwmhbo                       
7    wwmhbo                       
8    wwmhbo                       
9    wwmhbo                       
10   wwmhbo                       
11   wwmhbo                       
12   wwmhbo                       
13   wwmhbo                       
14   wwmhbo                       
15   wwmhbo                       
16   wwmhbo                       
17   wwmhbo                       
18   wwmhbo                       
19   wwmhbo                       
20   wwmhbo                       
21   wwmhbo                       
22   wwmhbo                       
23   wwmhbo                       
24   wwmhbo                       
25   wwmhbo                       
26   wwmhbo                       
27   wwmhbo                       
28   wwmhbo                       
29   wwmhbo                       
..      ...                       
228  wwmhbo                       
229  wwmhbo                       
230  wwmhbo                       
231  wwmhbo                       
232  wwmhbo                       
233  wwmhbo                       
234  wwmhbo                       
235  wwmhbo                       
236  wwmhbo                       
237  wwmhbo                       
238  wwmhbo                       
239  wwmhbo                       
240  wwmhbo                       
241  wwmhbo                       
242  wwmhbo                       
243  wwmhbo                       
244  wwmhbo                       
245  wwmhbo                       
246  wwmhbo                       
247  wwmhbo                       
248  wwmhbo                       
249  wwmhbo                       
250  wwmhbo                       
251  wwmhbo                       
252  wwmhbo                       
253  wwmhbo                       
254  wwmhbo                       
255  wwmhbo                       
256  wwmhbo                       
257  wwmhbo                       

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       arrangement_keywords  \
0    [Oefening, Progressies, Tussendominanten, Harmonie, Analyse popmuziek]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
1    [Theorie, MOOC, Bestuurskunde]                                                                                                                                                                                                                                        

In [93]:
duplicate_docs_df.groupby('arrangement_collection_name').count()

,arrangement_keywords,id,language,mime_type,pipeline,text,title,url
arrangement_collection_name,,,,,,,,
hbovpk,2,2,2,2,2,2,2,2
stimuleringsregeling,4,4,4,2,4,4,4,4
wwmhbo,252,252,252,252,252,55,252,252


It seems that most of these documentes are from wwmhbo.

In [102]:
print(len(duplicate_docs_df.groupby(['url', 'id']).count()))
print(len(duplicate_docs_df.groupby('id').count()))
print(len(duplicate_docs_df.groupby('url').count()))

101
101
101


And, indeed, they all share the same URL and ID

## 2. Verify that certain documents go through all stages
First we check if all the documents in the ranked queries are there

In [104]:
def read_queries(query_file):
    with open(query_file, 'rt') as file:
        return json.load(file)
QUERIES = {lang: read_queries(QUERY_FILES[lang]) for lang in LANGUAGES}

In [106]:
def are_queries_in_freeze(lang, queries):
    missing_docs = {}
    for query in queries:
        for doc in query['items']:
            if util.get_raw_document(doc['hash'], FREEZE_DATA_FOLDER) is None:
                missing_docs[doc['hash']] = {'query': query["queries"][0], 'lang': lang}
    return missing_docs

In [107]:
freeze_missing = [are_queries_in_freeze(lang, QUERIES[lang]) for lang in LANGUAGES]
freeze_missing

[{},
 {'aa508762def8fb9f34ed7e0186eca1c4207e7298': {'query': 'leren leren',
   'lang': 'nl'}}]

### Some useful functions

In [108]:
def get_identifier_freeze(id, identifier):
    """
    Finds a document with 'id' and retrieves the property 'identifier' or None.
    """
    if util.get_raw_document(id, FREEZE_DATA_FOLDER) is not None:
        return util.get_raw_document(id, FREEZE_DATA_FOLDER)[identifier]

## Check if queries are split correctly

To begin with we read in the queries and see:
1. The language of the raw document.
2. The text of the raw document.

In [111]:
query_split_view = [
    {
        'id': item['hash'],
        'freeze_lang': get_identifier_freeze(item['hash'], 'language')['from_text'] if get_identifier_freeze(item['hash'], 'language') is not None else '',
        'query_lang': lang,
        'title': get_identifier_freeze(item['hash'], 'title'),
        'text_snippet': get_identifier_freeze(item['hash'], 'text')[:100] if get_identifier_freeze(item['hash'], 'text') is not None else ''
    }
    for lang in LANGUAGES 
    for query in QUERIES[lang] 
    for item in query['items']
]

query_view = pd.DataFrame(query_split_view)

In [113]:
incorrect_lang = query_view[query_view['freeze_lang'] != query_view['query_lang']]
incorrect_lang

,freeze_lang,id,query_lang,text_snippet,title
64,,aa508762def8fb9f34ed7e0186eca1c4207e7298,nl,,None
93,en,d610580ea5cea9d1cc0889d9e52ef5add8c1fc31,nl,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n,Overzicht bereik en transposities veel voorkomende instrumenten
130,en,15350fb67a0634c6612e5a575fed67b75409e365,nl,\n\n,Didactiek van wiskundig denken
131,en,dc55a6bb1c525b3494c07d0557f557605ffec7c2,nl,\n\n,Didactiek van wiskundig denken


In [2]:
util.get_raw_document('34adeddc05423967fd2b70e1068581ba87c0d547', RAW_DATA_FOLDER)

{'id': '34adeddc05423967fd2b70e1068581ba87c0d547',
 'title': 'Water geven',
 'language': {'metadata': 'nl', 'from_text': 'nl', 'from_title': 'nl'},
 'url': 'https://maken.wikiwijs.nl/81245/PBAN7_Bedrijfsgebouwen_tuinbouw/item6f8ffcb899cc385a.html',
 'text': '\nWater geven\n\n\n\n\n\n\n',
 'mime_type': 'application/xhtml+xml',
 'pipeline': {'harvest': '5a499c4'},
 'arrangement_keywords': [],
 'arrangement_collection_name': 'wwmhbo'}

## Check if queries are in correct ES folder
Given that the queries have been split correctly to languages, we check the corresponding elasticsearch documents.

In [124]:
def are_queries_in_es(lang, queries, index):
    missing_docs = {}
    for query in queries:
        for doc in query['items']:
            try:
                response = es_client.get(index=index, doc_type=DOC_TYPE, id=doc['hash'])
            except Exception as e:
                missing_docs[doc['hash']] = {'query': query["queries"][0], 'lang': lang}
    return missing_docs

In [125]:
es_missing = [are_queries_in_es(lang, QUERIES[lang], INDIECES[lang]) for lang in LANGUAGES]
es_missing

[2019-03-14 14:45:09,916][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-en/_doc/18604a456db12d50da67af95f596ab795ea4eb1a [status:200 request:0.006s]
[2019-03-14 14:45:09,921][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-en/_doc/8c87b0c6111df4d2fe8633a8b274851b2ec4a2c0 [status:200 request:0.004s]
[2019-03-14 14:45:09,927][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-en/_doc/fe118ba3f8b3d49a0152f7e1f1bdfbb31490b808 [status:200 request:0.006s]
[2019-03-14 14:45:09,932][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-en/_doc/eb6c2ae9d0f29b7de72a6dc6acfa96a3dcff111f [status:200 request:0.005s]
[2019-03-14 14:45:09,936][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-en/_doc/a000df1ed8e28fc1f9458231ac9fe60c0debe396 [status:200 request:0.003s]
[2019-03-14 14:45:09,941][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-en/_doc/18604a456db12d50da67af95f596ab7

[2019-03-14 14:45:10,179][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-en/_doc/eed8d23e4f4b142bf04a9a962ab4aef6e390a3f1 [status:200 request:0.004s]
[2019-03-14 14:45:10,183][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-en/_doc/cf92f2726f45908b33d006de64d5f42153dcc1ca [status:200 request:0.003s]
[2019-03-14 14:45:10,187][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-en/_doc/f5a6dbdd62e9e329ca92066ea1a592defd5c7f04 [status:200 request:0.004s]
[2019-03-14 14:45:10,191][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-en/_doc/eef2df3658e0eea6da1bf3b778a05242601f8ac5 [status:200 request:0.003s]
[2019-03-14 14:45:10,195][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-en/_doc/401035affeb86d1dcba441593b2ff22d78e9910a [status:200 request:0.003s]
[2019-03-14 14:45:10,204][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-en/_doc/330d36593b97ec9864b9557f8a0a4b3

[2019-03-14 14:45:10,669][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-nl/_doc/9699dc0be2e54f3d59ced179c30ab7bf7ef19618 [status:200 request:0.003s]
[2019-03-14 14:45:10,674][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-nl/_doc/26a14ae72d8aa1c015510d55e500fe869f426d12 [status:200 request:0.004s]
[2019-03-14 14:45:10,691][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-nl/_doc/0dd9e946f3aca97d3e7f09df68efc35a114674e7 [status:200 request:0.016s]
[2019-03-14 14:45:10,708][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-nl/_doc/eff32a56fad71d67afe3d4fbcaffa42e1a9135f6 [status:200 request:0.015s]
[2019-03-14 14:45:10,717][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-nl/_doc/4c89bc1b0f5c76b6b58fbd13e590c0921a9c42f8 [status:200 request:0.006s]
[2019-03-14 14:45:10,724][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-nl/_doc/abb17d5655caa8c1be59e9ed278ad30

[2019-03-14 14:45:10,960][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-nl/_doc/9da5ce551c7d507e4df8429daf7f08952ddd28cf [status:200 request:0.003s]
[2019-03-14 14:45:10,964][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-nl/_doc/cd27582428028148af2ae465cf03175314b45f81 [status:200 request:0.003s]
[2019-03-14 14:45:10,970][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-nl/_doc/096e5cd027af3a71586ab0d80ad0f60ab7d83724 [status:200 request:0.006s]
[2019-03-14 14:45:10,976][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-nl/_doc/e8064096f14ec6d252ba5bccdedcddd9ef1a6734 [status:200 request:0.004s]
[2019-03-14 14:45:10,992][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-nl/_doc/c72334bcb5f4c517ecacdb76b0654c2725c47ff9 [status:200 request:0.014s]
[2019-03-14 14:45:10,996][INFO   ] GET https://surfpol.sda.surf-hosted.nl:443/freeze4_no_filter-nl/_doc/6f6ae1d8cd6a2cc541620ded919f9f3

[{},
 {'aa508762def8fb9f34ed7e0186eca1c4207e7298': {'query': 'leren leren',
   'lang': 'nl'},
  'd610580ea5cea9d1cc0889d9e52ef5add8c1fc31': {'query': 'noteren van muziek',
   'lang': 'nl'},
  '15350fb67a0634c6612e5a575fed67b75409e365': {'query': 'didactiek',
   'lang': 'nl'},
  'dc55a6bb1c525b3494c07d0557f557605ffec7c2': {'query': 'didactiek',
   'lang': 'nl'}}]

These should be the same documents as above.

In [33]:
def explain(id, query):
    print(es_client.explain(index=INDEX, doc_type=DOC_TYPE, id=id, query=query))

In [36]:
def search(query, **kwargs):
    pprint.pprint(es_client.search(index=INDEX, doc_type=DOC_TYPE, body=query, **kwargs))

In [37]:
usersSearch = 'injecties geven'
Fields = ['title^2', 'text']

query = {
    'query': {
        'multi_match': { 
            'query': usersSearch, #A
            'fields': Fields, #B
        },
    },
  #  'size': '100'
}
search(query)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '34adeddc05423967fd2b70e1068581ba87c0d547',
                    '_index': 'freeze3-2-nl',
                    '_score': 15.291727,
                    '_source': {'arrangement_collection_name': 'wwmhbo',
                                'conformed_mime_type': 'html',
                                'keywords': [],
                                'mime_type': 'application/xhtml+xml',
                                'text': ' Water geven ',
                                'text_plain': ' Water geven ',
                                'title': 'Water geven',
                                'title_plain': 'Water geven',
                                'url': 'https://maken.wikiwijs.nl/81245/PBAN7_Bedrijfsgebouwen_tuinbouw/item6f8ffcb899cc385a.html'},
                    '_type': '_doc'},
                   {'_id': '41e6906f81c934c40083ea947f5e029007ab5600',
                    '_index': 'freeze3

                                        'dan de MantelScan (DOC15.4) af bij '
                                        'iemand die je in je eigen omgeving '
                                        'kent die mantelzorger is. Wanneer je '
                                        'dit hebt gedaan heb je veel '
                                        'informatie over de zorgsituatie '
                                        'gekregen. Ook de knelpunten zijn '
                                        'wellicht naar boven gekomen. Ook heb '
                                        'je door de MantelScan inzicht '
                                        'gekregen in de ervaren belasting van '
                                        'de mantelzorger. De MantelScan bevat '
                                        'namelijk 2 vragenlijsten die ook in '
                                        'Tabel 15.3 zijn behandeld (de SRBS en '
                                        'de EDIZ). Bedenk welk gerich

In [ ]:
raw

In [72]:
explain(id='3605374af3eedfa7084c17cbea6d0fce3862bfb0')

GET https://surfpol.sda.surf-hosted.nl:443/freeze3/doc/3605374af3eedfa7084c17cbea6d0fce3862bfb0/_explain [status:400 request:0.047s]


RequestError: RequestError(400, 'action_request_validation_exception', 'Validation Failed: 1: query is missing;')